<a href="https://colab.research.google.com/github/KevinTheRainmaker/High_Performance_Computing/blob/main/matrix_multiplication_GPU_vs_CPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-kvuvf_jz
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-kvuvf_jz
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4306 sha256=947c707669dd521d28e293baef4d947de23246f03e82de349c9ba8fc32f5cb1f
  Stored in directory: /tmp/pip-ephem-wheel-cache-dw1ax4b3/wheels/c5/2b/c0/87008e795a14bbcdfc7c846a00d06981916331eb980b6c8bdf
Successfully built NVCCPlugin


In [3]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [15]:
%%cu

#include <stdio.h>
#include <stdlib.h>
#include <assert.h>
#include<time.h>

#define BLOCK_SIZE 16

//GPU Ver.
__global__ void gpu_matrix_mult(int *d_a, int *d_b, int *d_result, int m){
    __shared__ int tile_a[BLOCK_SIZE][BLOCK_SIZE];
    __shared__ int tile_b[BLOCK_SIZE][BLOCK_SIZE];

    int row = blockIdx.y * BLOCK_SIZE + threadIdx.y;
    int col = blockIdx.x * BLOCK_SIZE + threadIdx.x;
    int tmp = 0;
    int idx;

    for (int sub = 0; sub < gridDim.x; ++sub){
        idx = row * m + sub * BLOCK_SIZE + threadIdx.x;

        if(idx >= m*m){
            tile_a[threadIdx.y][threadIdx.x] = 0;
        }
        else{
            tile_a[threadIdx.y][threadIdx.x] = d_a[idx];
        }

        idx = (sub * BLOCK_SIZE + threadIdx.y) * m + col;

        if(idx >= m*m){
            tile_b[threadIdx.y][threadIdx.x] = 0;
        }
        else{
            tile_b[threadIdx.y][threadIdx.x] = d_b[idx];
        }

        __syncthreads();

        for (int k = 0; k < BLOCK_SIZE; ++k){
            tmp += tile_a[threadIdx.y][k] * tile_b[k][threadIdx.x];
        }

        __syncthreads();
    }

    if(row < m && col < m){
        d_result[row * m + col] = tmp;
    }
}

// CPU Ver.
void cpu_matrix_mult(int *h_a, int *h_b, int *h_result, int m) {
    for (int i = 0; i < m; ++i){
        for (int j = 0; j < m; ++j){
            int tmp = 0.0;
            for (int h = 0; h < m; ++h){
                tmp += h_a[i * m + h] * h_b[h * m + j];
            }
            h_result[i * m + j] = tmp;
        }
    }
}

int main(int argc, char const *argv[]){
    int m = 1024;
    srand((unsigned)time(NULL));

    // allocate device memory 
    int *h_a, *h_b, *h_c, *h_cc;
    cudaMallocHost((void **) &h_a, sizeof(int)*m*m);
    cudaMallocHost((void **) &h_b, sizeof(int)*m*m);
    cudaMallocHost((void **) &h_c, sizeof(int)*m*m);
    cudaMallocHost((void **) &h_cc, sizeof(int)*m*m);

    // random initialize matrix A
    for (int i = 0; i < m; ++i) {
        for (int j = 0; j < m; ++j) {
            h_a[i * m + j] = rand() % 1024;
        }
    }

    // random initialize matrix B
    for (int i = 0; i < m; ++i) {
        for (int j = 0; j < m; ++j) {
            h_b[i * m + j] = rand() % 1024;
        }
    }

    float gpu_elapsed_time_ms, cpu_elapsed_time_ms;

    // events to count the execution time
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // start to count execution time of GPU version
    cudaEventRecord(start, 0);
    
    // allocate device memory 
    int *d_a, *d_b, *d_c;
    cudaMalloc((void **) &d_a, sizeof(int)*m*m);
    cudaMalloc((void **) &d_b, sizeof(int)*m*m);
    cudaMalloc((void **) &d_c, sizeof(int)*m*m);

    // transfer data from host to device memory
    cudaMemcpy(d_a, h_a, sizeof(int)*m*m, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, sizeof(int)*m*m, cudaMemcpyHostToDevice);

    unsigned int grid_rows = (m + BLOCK_SIZE - 1) / BLOCK_SIZE;
    unsigned int grid_cols = (m + BLOCK_SIZE - 1) / BLOCK_SIZE;
    dim3 dimGrid(grid_cols, grid_rows);
    dim3 dimBlock(BLOCK_SIZE, BLOCK_SIZE);
   
    // call function: GPU
    gpu_matrix_mult<<<dimGrid, dimBlock>>>(d_a, d_b, d_c, m);

    // transfer data from device to host 
    cudaMemcpy(h_c, d_c, sizeof(int)*m*m, cudaMemcpyDeviceToHost);
    cudaThreadSynchronize();

    // terminate counting time
    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);

    // compute time on GPU
    cudaEventElapsedTime(&gpu_elapsed_time_ms, start, stop);
    printf("Elapsed time on matrix multiplication of A(%dx%d) X B(%dx%d) on GPU: %f ms.\n\n", m, m, m, m, gpu_elapsed_time_ms);

    // start to count execution time of CPU version
    cudaEventRecord(start, 0);

    // call function: CPU
    cpu_matrix_mult(h_a, h_b, h_cc, m);

    // terminate counting
    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);

    // compute time on CPU
    cudaEventElapsedTime(&cpu_elapsed_time_ms, start, stop);
    printf("Elapsed time on matrix multiplication of A(%dx%d) X B(%dx%d) on CPU: %f ms.\n\n", m, m, m, m, cpu_elapsed_time_ms);

    // validate results computed by GPU
    int same_result = 1;
    for (int i = 0; i < m; ++i){
        for (int j = 0; j < m; ++j){            
            if(h_cc[i*m + j] != h_c[i*m + j]){
                same_result = 0;
            }
        }
    }

    // roughly compute speedup
    if(same_result){
        printf("Result of GPU and CPU are same.\n\nGPU speedup over CPU: %.2fx", cpu_elapsed_time_ms / gpu_elapsed_time_ms);
    }
    else{
        printf("Results are not same.\n");
    }

    // deallocate device memory
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    cudaFreeHost(h_a);
    cudaFreeHost(h_b);
    cudaFreeHost(h_c);
    cudaFreeHost(h_cc);
    return 0;
}

Elapsed time on matrix multiplication of A(1024x1024) X B(1024x1024) on GPU: 7.349728 ms.

Elapsed time on matrix multiplication of A(1024x1024) X B(1024x1024) on CPU: 8101.461426 ms.

Result of GPU and CPU are same.

GPU speedup over CPU: 1102.28x
